In [ ]:
# https://jsonplaceholder.typicode.com/
# activate service account api
gcloud services enable dataflow.googleapis.com
# create cloud storage bucket
gsutil mb  gs://$PROJECTID
# create cloud storage bucket with location flag
gsutil mb -l us gs://qwiklabs-gcp-04-bad8daf3b464-us
gsutil mb -l eu gs://qwiklabs-gcp-04-bad8daf3b464-eu
# create dataset in bigquery
bq mk --dataset $PROJECTID:DATASETID
# create dataset in bigquery with location flag
bq mk --location=EU --dataset qwiklabs-gcp-04-bad8daf3b464:nyc_tlc_EU
# create table in bigquery
bq mk --table $PROJECTID:DATASETID.TABLEID ./schema.json
# clone bq table
bq cp --clone -n DATASETID.TABLEID DATASETID.NEWTABLEID

In [ ]:
gcloud composer environments run composer-advanced-lab \
--location us-east1 variables -- \
set table_list_file_path /home/airflow/gcs/dags/bq_copy_eu_to_us_sample.csv


# create vertex ai custom training service account
SERVICE_ACCOUNT_ID=vertex-custom-training-sa
gcloud iam service-accounts create $SERVICE_ACCOUNT_ID  \
    --description="A custom service account for Vertex custom training with Tensorboard" \
    --display-name="Vertex AI Custom Training"
    
    
# grant service account access to storage
PROJECT_ID=$(gcloud config get-value core/project)
gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:$SERVICE_ACCOUNT_ID@$PROJECT_ID.iam.gserviceaccount.com \
    --role="roles/storage.admin"
    
# grant service account access to bigquery
gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:$SERVICE_ACCOUNT_ID@$PROJECT_ID.iam.gserviceaccount.com \
    --role="roles/bigquery.admin"
    
# grant service account access to vertex ai for model training and deployment
gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:$SERVICE_ACCOUNT_ID@$PROJECT_ID.iam.gserviceaccount.com \
    --role="roles/aiplatform.user"